<a href="https://colab.research.google.com/github/smf-9000/kaggle/blob/main/%5B003%5D_tps_aug_2021_%5Blgbm_xgb_catb_combination%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%config Completer.use_jedi = False

In [ ]:
# !pip3 show lightgbm

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, imread

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

from statsmodels.nonparametric.smoothers_lowess import lowess

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

import warnings

# from bayes_opt import BayesianOptimization

import optuna

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/003-tps/ridge_y_df.csv
/kaggle/input/003-tps/ridge_train_df.csv
/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2021/train.csv
/kaggle/input/tabular-playground-series-aug-2021/test.csv


In [ ]:
submission_ex = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/sample_submission.csv')
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv')
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/test.csv')

In [ ]:
targets_df = train_df[['loss']].copy()
train_df.drop(['loss'], axis=1, inplace=True)

In [ ]:
from sklearn.cluster import KMeans

n_clusters = 2

all_df = pd.concat([train_df, test_df])
data = all_df.values
km = KMeans(n_clusters = n_clusters)
km.fit(data)
k_clus = km.labels_

# one_h_clus = np.zeros((k_clus.size, k_clus.max()+1))
# one_h_clus[np.arange(k_clus.size), k_clus] = 1
# for i in range(n_clusters):
#     all_df['clus_' + str(i)] = one_h_clus[:,i]

# all_df['k_clus'] = k_clus
_dist = km.transform(data)
_dict = {'dist_from_' + str(i): _dist[:,i] for i in range(n_clusters)}
for k, v in _dict.items():
    all_df[k] = v
train_df, test_df = all_df.iloc[:train_df.shape[0],:].copy(), all_df.iloc[-test_df.shape[0]:,:].copy()

In [ ]:
# pd.value_counts(targets_df.loss)

In [ ]:
# pd.value_counts(train_df.k_clus)

In [ ]:
# pd.value_counts(test_df.k_clus)

In [ ]:
train_df.drop(['id'], axis=1, inplace=True) 

In [ ]:
# warnings.filterwarnings("ignore")

# # ################################################################
# fig = plt.figure(figsize = (30,30))
# ax = fig.gca()
# hist = train_df.hist(bins=50, ax = ax)

In [ ]:
# for c in train_df.columns:
#     if c != 'id':
#         fig = plt.figure(figsize = (30, 3))
#         g = train_df.iloc[:500,:][c].plot()
#         g = targets_df.iloc[:500,:]['loss'].plot()
#         g.legend([c])

In [ ]:
# warnings.filterwarnings("ignore")

# # ################################################################
# fig = plt.figure(figsize = (30,10))
# ax = fig.gca()
# hist = targets_df[targets_df.columns.values].hist(bins=84, ax = ax)

In [ ]:
# warnings.filterwarnings("ignore")

# train_without_zero_loss_df = train_df[targets_df['loss'] != 0]

# # ################################################################
# fig, axes = plt.subplots(nrows = 10, ncols = 10, figsize=(30, 30))
# i = 0
# for c in train_df.columns[1:]:
#     ax = axes[i // 10, i % 10]
#     ax.set_title(c)
#     train_df[c].hist(bins=50, ax = ax)
#     train_without_zero_loss_df[c].hist(bins=50, ax = ax, alpha = 0.7)
#     i += 1

### feature correlation

In [ ]:
# qq_df = pd.concat([train_df, targets_df], axis=1, join='inner')

# # ################################################################
# corr = qq_df.corr()

# mask = np.zeros_like(corr)
# mask[np.triu_indices_from(mask)] = True

# fig = plt.figure(figsize = (25,25))
# sns.heatmap(corr, cmap="flare", mask=mask)
# plt.show()

In [ ]:
X = train_df
y = targets_df[['loss']]

In [ ]:
kfolds = KFold(n_splits = 5, shuffle = True) # , random_state = 37

In [ ]:
def tune(objective):
    study = optuna.create_study(direction = "minimize")
    study.optimize(objective, n_trials = 1000, timeout = 3*60*60) # "specifying 'timeout' option to Study.optimize() instead of 'n_trials' and set the timeout value as long as time allows?"
    optuna.visualization.plot_optimization_history(study)
    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score} \nOptimized parameters: params = {params}")
    return params

In [ ]:
def lightgbm_objective(trial):
    
    params = {
        "objective": "rmse",
        "metric": "rmse",
        "boosting_type": "gbdt",
        'n_estimators': trial.suggest_int("n_estimators", 100, 1000),
        'max_depth': trial.suggest_int("max_depth", -1, 100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 1),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 1),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 100),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 0, 5),
#         'device': 'gpu',
#         'gpu_platform_id': 0,
#         'gpu_device_id': 0
    }
    
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'rmse', valid_name='valid_0')
    
    model = lgb.LGBMRegressor(**params,
                              n_jobs=-1,
                              random_state = 37) #,
                              # device_type="gpu")
    
    val_rmse = []
    for train_index, test_index in kfolds.split(X):
 
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
        model.fit(
            X_train, 
            y_train, 
            eval_metric = "rmse", 
            eval_set = [(X_val, y_val)], #(X_train, y_train), 
            verbose = 100,
#             early_stopping_rounds = 20,
            callbacks = [pruning_callback])
        oof_pred1 = model.predict(X_val)
        oof_pred1 = np.clip(oof_pred1, y['loss'].min(), y['loss'].max())
        val_rmse.append(mean_squared_error(y_val, oof_pred1, squared = False))
    
   
    score = sum(val_rmse) / len(val_rmse)
    
    return score
    

In [ ]:
# lightgbm_params = tune(lightgbm_objective)

In [ ]:
def xgb_objective(trial):
    
    params = {
        "verbosity": 0,
        "objective": "reg:squarederror",
        "tree_method": "gpu_hist", # 
        "booster": "gbtree",
        "lambda": trial.suggest_float("lambda", 1, 10),
        "alpha": trial.suggest_float("alpha", 0, 10),
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        'max_depth': trial.suggest_int("max_depth", 2, 20),
        'n_estimators': trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    
    
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation_0-rmse')
    
    model = xgb.XGBRegressor(**params,
                             n_jobs = 4,
                             random_state = 37)

    val_rmse = []
    for train_index, test_index in kfolds.split(X):
 
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    
        model.fit(
            X_train, 
            y_train, 
            eval_metric="rmse", 
            eval_set=[(X_val, y_val)], # (X_train, y_train), 
            verbose = 100,
#             early_stopping_rounds = 100,
            callbacks = [pruning_callback])
    
        oof_pred2 = model.predict(X_val)
        oof_pred2 = np.clip(oof_pred2, y['loss'].min(), y['loss'].max())
        val_rmse.append(mean_squared_error(y_val, oof_pred2, squared = False))
   
    score = sum(val_rmse) / len(val_rmse)
    
    return score

In [ ]:
# xgb_params = tune(xgb_objective)

In [ ]:
def catb_objective(trial):
    
    params = {
        "verbose": 100,
        'grow_policy': 'Depthwise',
        'loss_function': 'RMSE',
        'eval_metric': 'RMSE',
        'od_type': 'Iter',
        'task_type': 'GPU',
        "random_strength": trial.suggest_float("random_strength", 1.0, 5),
        'min_child_samples': trial.suggest_int("min_child_samples", 1, 35),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 20),
        'depth': trial.suggest_int("depth", 2, 15),
        'iterations': trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1)
    }
    
    model = CatBoostRegressor(**params,
                             thread_count = 4,
                             random_state = 37)

    val_rmse = []
    for train_index, test_index in kfolds.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[test_index]
        y_train, y_val = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(
                    X_train, 
                    y_train,
                    eval_set=[(X_val, y_val)],
#                     early_stopping_rounds = 100
                    )
        
        oof_pred3 = model.predict(X_val)
        oof_pred3 = np.clip(oof_pred3, y['loss'].min(), y['loss'].max())
        val_rmse.append(mean_squared_error(y_val, oof_pred3, squared = False))
   
    score = sum(val_rmse) / len(val_rmse)

    return score

In [ ]:
# catb_params = tune(catb_objective)

### #data

In [ ]:
_target = 'loss'
y = np.squeeze(np.array(targets_df[[_target]].values, dtype='float32'))
X_test = np.array(test_df.iloc[:,1:].values, dtype='float32')

X_train, X_val, y_train, y_val = train_test_split(train_df, y, test_size = 0.5, random_state = 42, shuffle = True)
# split_train = 125000
# X_train, X_val = train_df.iloc[:split_train,:], train_df.iloc[split_train:, :]
# y_train, y_val = y[:split_train], y[split_train:]

ridge_train_df = None
ridge_y_df = None
ridge_final_df = None
p1, p2, p3 = None, None, None

In [ ]:
submit = True

### #lightGBM

In [ ]:
lightgbm_params = {'n_estimators': 535, 
                   'max_depth': 65, 
                   'learning_rate': 0.037982101708680834, 
                   'lambda_l1': 1.4182871983597388e-07, 
                   'lambda_l2': 0.3020801492297757, 
                   'num_leaves': 45, 
                   'min_child_samples': 75, 
                   'feature_fraction': 0.4537941150166718, 
                   'bagging_fraction': 0.5661024435645774, 
                   'bagging_freq': 0}

lightgbm_params['n_estimators'] = 15000
lightgbm_params['learning_rate'] = 0.0037982101708680834

lgbm_reg = lgb.LGBMRegressor(**lightgbm_params, 
                       objective = 'rmse',
                       metric = 'rmse',
                       boosting_type = 'gbdt',
                       n_jobs=-1)

In [ ]:
if not submit:
    lgbm_reg.fit(
        X_train, 
        y_train,
        eval_set = [(X_val, y_val)],
        verbose = 100,
        early_stopping_rounds = 500)
    oof_pred_1 = lgbm_reg.predict(X_val)
    oof_pred_1 = np.clip(oof_pred_1, y.min(), y.max())

    print('target: ', _target)
    print('=================================')
    print('rmse: ', mean_squared_error(y_val, oof_pred_1, squared = False))
    
    print('Plotting feature importances...')
    fea_imp = pd.DataFrame({'imp': lgbm_reg.feature_importances_, 'col': X_train.columns})
    fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False])#.iloc[-10:]
    fea_imp.plot(kind='barh', x='col', y='imp', figsize=(20, 30), legend=None)
    plt.title('LGBMRegressor - Feature Importance')
    plt.ylabel('Features')
    plt.xlabel('Importance')
    
    ridge_train_df = pd.DataFrame({'oof_1': oof_pred_1})
else:
    print('submit s_1')
    submission_1 = test_df[['id']].copy()
    lgbm_reg.fit(train_df, y)
    
    print('prediction s_1')
    p1 = lgbm_reg.predict(X_test)
    p1 = np.clip(p1, y.min(), y.max())

    submission_1[_target] = p1
    submission_1.to_csv('submission_1.csv', index=False)
    test = pd.read_csv('submission_1.csv')
    print(test)

submit s_1
[LightGBM] [Warning] lambda_l1 is set=1.4182871983597388e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.4182871983597388e-07
[LightGBM] [Warning] feature_fraction is set=0.4537941150166718, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4537941150166718
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] lambda_l2 is set=0.3020801492297757, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.3020801492297757
[LightGBM] [Warning] bagging_fraction is set=0.5661024435645774, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5661024435645774
prediction s_1
            id      loss
0       250000  6.839050
1       250001  6.744543
2       250002  6.706794
3       250003  6.865972
4       250004  6.920852
...        ...       ...
149995  399995  6.806974
149996  399996  6.825065
149997  399997  6.702042
149998  399998  6.767999
149999  399999 

### # XGB

In [ ]:
xgb_params = {'lambda': 6.699320718487671, 
              'alpha': 7.10622549536764, 
              'subsample': 0.6602661885588786, 
              'colsample_bytree': 0.919626651702815, 
              'max_depth': 3, 
              'n_estimators': 497, 
              'learning_rate': 0.09813314953213292, 
              'min_child_weight': 194}

xgb_params['n_estimators'] = 10500
xgb_params['learning_rate'] = 0.009813314953213292

xgb_reg = xgb.XGBRegressor(
                            **xgb_params,
                            verbosity = 0,
                            objective = "reg:squarederror",
                            tree_method = "hist", # gpu_
                            booster = "gbtree")

In [ ]:
if not submit:
    xgb_reg.fit(
                X_train, 
                y_train, 
                eval_metric="rmse", 
                eval_set=[(X_val, y_val)], 
                verbose=100, 
                early_stopping_rounds = 500)

    oof_pred_2 = xgb_reg.predict(X_val)
    oof_pred_2 = np.clip(oof_pred_2, y.min(), y.max())
    
    print('target: ', _target)
    print('=================================')
    print('rmse: ', mean_squared_error(y_val, oof_pred_2, squared = False))
    
    print('Plotting feature importances...')
    fea_imp = pd.DataFrame({'imp': xgb_reg.feature_importances_, 'col': X_train.columns})
    fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False])#.iloc[-10:]
    fea_imp.plot(kind='barh', x='col', y='imp', figsize=(20, 30), legend=None)
    plt.title('XGBRegressor - Feature Importance')
    plt.ylabel('Features')
    plt.xlabel('Importance')
    
    ridge_train_df['oof_2'] = oof_pred_2
else:
    print('submit s_2')
    submission_2 = test_df[['id']].copy()
    xgb_reg.fit(train_df, y)
    
    print('prediction s_2')
    p2 = xgb_reg.predict(X_test)
    p2 = np.clip(p2, y.min(), y.max())

    submission_2[_target] = p2
    submission_2.to_csv('submission_2.csv', index=False)
    test = pd.read_csv('submission_2.csv')
    print(test)

submit s_2
prediction s_2
            id      loss
0       250000  4.405971
1       250001  4.363369
2       250002  4.152820
3       250003  4.538853
4       250004  4.771395
...        ...       ...
149995  399995  4.328613
149996  399996  4.374373
149997  399997  4.364668
149998  399998  4.406545
149999  399999  4.513653

[150000 rows x 2 columns]


### #catB

In [ ]:
catb_params = {'random_strength': 2.3629132213017803, 
               'min_child_samples': 13, 
               'l2_leaf_reg': 8.840092897150026, 
               'depth': 6, 
               'iterations': 791, 
               'learning_rate': 0.03419185967256572}


catb_params['iterations'] = 15000
catb_params['learning_rate'] = 0.003419185967256572

catb_reg = CatBoostRegressor(
                        **catb_params,
                        verbose = 100,
                        grow_policy = 'Depthwise',
                        loss_function = 'RMSE',
                        eval_metric = 'RMSE',
                        od_type = 'Iter',
                        # task_type: 'GPU'
                        )

In [ ]:
if not submit:
    
    catb_reg.fit(
                X_train, 
                y_train,
                eval_set=[(X_val, y_val)],
                verbose=100, 
                early_stopping_rounds = 500)
    oof_pred_3 = catb_reg.predict(X_val)
    oof_pred_3 = np.clip(oof_pred_3, y.min(), y.max())

    print('Plotting feature importances...')
    fea_imp = pd.DataFrame({'imp': catb_reg.feature_importances_, 'col': X_train.columns})
    fea_imp = fea_imp.sort_values(['imp', 'col'], ascending=[True, False])#.iloc[-10:]
    fea_imp.plot(kind='barh', x='col', y='imp', figsize=(20, 30), legend=None)
    plt.title('CatBoost - Feature Importance')
    plt.ylabel('Features')
    plt.xlabel('Importance')

    ridge_train_df['oof_3'] = oof_pred_3
else:
    print('submit s_3')
    submission_3 = test_df[['id']].copy()
    catb_reg.fit(train_df, y)
    
    print('prediction s_3')
    p3 = catb_reg.predict(X_test)
    p3 = np.clip(p3, y.min(), y.max())

    submission_3[_target] = p3
    submission_3.to_csv('submission_3.csv', index=False)
    test = pd.read_csv('submission_3.csv')
    print(test)

submit s_3
0:	learn: 7.9400499	total: 183ms	remaining: 18.1s
99:	learn: 7.9311484	total: 18.2s	remaining: 0us
prediction s_3
            id      loss
0       250000  6.773577
1       250001  6.740703
2       250002  6.705142
3       250003  6.865776
4       250004  6.944710
...        ...       ...
149995  399995  6.761159
149996  399996  6.780478
149997  399997  6.766689
149998  399998  6.788790
149999  399999  6.866636

[150000 rows x 2 columns]


### #ridge

In [ ]:
if not submit:
    ridge_train_df.to_csv('ridge_train_df.csv', index = False)
    ridge_y_df = pd.DataFrame({'loss': y_val})
    ridge_y_df.to_csv('ridge_y_df.csv', index = False)
else:
    ridge_train_df = pd.read_csv('/kaggle/input/003-tps/ridge_train_df (1).csv')
    ridge_y_df = pd.read_csv('/kaggle/input/003-tps/ridge_y_df (1).csv')
    ridge_final_df = pd.DataFrame({'_1': p1})
    ridge_final_df['_2'] = p2
    ridge_final_df['_3'] = p3

In [ ]:
if not submit:
    steps = [
        ('poly', PolynomialFeatures()), # degree = 2
        ('model', Ridge())
    ]
    model = Pipeline(steps)
    parameters = [ {'model__alpha': np.array([450, 400, 375, 350, 225, 300, 250]), 'poly__degree': np.array([1, 2])} ]

    grid_search = GridSearchCV(estimator = model, 
                               param_grid = parameters,
                               scoring = 'neg_mean_squared_error',
                               cv = 10,
                               n_jobs = -1)
    grid_search = grid_search.fit(ridge_train_df, ridge_y_df)
    print(grid_search.best_params_)
    model = grid_search.best_estimator_
    ridge_pred = model.predict(ridge_train_df)
    print(ridge_pred)
else:
    steps = [
        ('poly', PolynomialFeatures(degree = 2)),
        ('model', Ridge(alpha = 350))
    ]
    model = Pipeline(steps)
    model.fit(ridge_train_df, ridge_y_df)
    ridge_pred = model.predict(ridge_final_df)
    print(ridge_pred)
    submission_ = test_df[['id']].copy()
    submission_['loss'] = ridge_pred
    submission_.to_csv('submission_.csv', index=False)

[[5.58646403]
 [5.52128681]
 [5.3659759 ]
 ...
 [5.50102088]
 [5.55443808]
 [5.63266142]]
